In [1]:
# Agent for studzee application. [ Testing with ML PDF ]
# Agent Architecture :
# 1. PDF Files
# 2. Documet Loader
# 3. Text Splitter
# 4. Embeddings
# 5. Vector Store
# 6. Retriever
# 7. Agent 
# 8. Structured JSON Output 

# This agent will work like : it will take user input which is some content and will generate a response based on the PDF files.

In [2]:
# Ollama for local 
OLLAMA_LLM_MODEL = "llama3.2:latest"
OLLAMA_EMBEDDING_MODEL = "mxbai-embed-large"

# AWS Bedrock for production
BEDROCK_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
BEDROCK_EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"
REGION = "us-east-1"


In [4]:
from langchain_community.document_loaders import PyPDFLoader
import pprint 

file_path = "E:\\FURFROU\\notebooks\\.docs\\Activation Functions.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()
pprint.pp(docs[0].metadata)


{'producer': 'pdfTeX-1.40.27',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2026-01-15T19:20:05+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2026-01-15T19:20:05+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live '
                    '2025) kpathsea version 6.4.1',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'E:\\FURFROU\\notebooks\\.docs\\Activation Functions.pdf',
 'total_pages': 10,
 'page': 0,
 'page_label': '1'}


In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter( chunk_size = 800 , chunk_overlap = 150)
chunks = splitter.split_documents(docs)
pprint.pp(chunks[0].metadata)

{'producer': 'pdfTeX-1.40.27',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2026-01-15T19:20:05+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2026-01-15T19:20:05+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live '
                    '2025) kpathsea version 6.4.1',
 'subject': '',
 'title': '',
 'trapped': '/False',
 'source': 'E:\\FURFROU\\notebooks\\.docs\\Activation Functions.pdf',
 'total_pages': 10,
 'page': 0,
 'page_label': '1'}


In [ ]:
#  next 